In [105]:
import pandas as pd
df = pd.read_csv("data/invalid처리한_조합만들기데이터.csv",encoding='cp949')
df = df.drop('Unnamed: 0',1)

C:\Users\ice_9\AppData\Local\Temp\ipykernel_5844\1817518560.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed: 0',1)


In [106]:
bot = df[df['라인']  == 'BOTTOM']
sup = df[df['라인']  == 'UTILITY']
mid = df[df['라인']  == 'MIDDLE']
jungle = df[df['라인']  == 'JUNGLE']
top = df[df['라인']  == 'TOP']

In [107]:
bot_sup = pd.merge(bot,sup,how='inner',on=['GameID','teamId'])
mid_jug = pd.merge(mid,jungle,how='inner',on=['GameID','teamId'])
top_jug = pd.merge(top,jungle,how='inner',on=['GameID','teamId'])

In [108]:
bot_sup.drop(['win_x','라인_x','라인_y'],1, inplace=True)
mid_jug.drop(['win_x','라인_x','라인_y'],1, inplace=True)
top_jug.drop(['win_x','라인_x','라인_y'],1, inplace=True)

C:\Users\ice_9\AppData\Local\Temp\ipykernel_5844\2317211115.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  bot_sup.drop(['win_x','라인_x','라인_y'],1, inplace=True)
C:\Users\ice_9\AppData\Local\Temp\ipykernel_5844\2317211115.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  mid_jug.drop(['win_x','라인_x','라인_y'],1, inplace=True)
C:\Users\ice_9\AppData\Local\Temp\ipykernel_5844\2317211115.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  top_jug.drop(['win_x','라인_x','라인_y'],1, inplace=True)


In [109]:
bot_sup.columns=['GameID','teamId','원딜','승패','서폿']
mid_jug.columns=['GameID','teamId','미드','승패','정글']
top_jug.columns=['GameID','teamId','탑','승패','정글']

In [110]:
bot_sup = bot_sup.loc[:,['GameID','teamId','원딜','서폿','승패']]
mid_jug = mid_jug.loc[:,['GameID','teamId','미드','정글','승패']]
top_jug = top_jug.loc[:,['GameID','teamId','탑','정글','승패']]

In [116]:
# match_count = df['GameID'].count() / 2 # 한 게임당 두 행으로 구성되어 있으므로 2로 나눈다.
#바텀 조합
duo_stats1 = pd.DataFrame()
for carry in bot_sup['원딜'].unique(): # 각 원딜과 서포터마다 계산
    for support in bot_sup['서폿'].unique():
        condition =  (bot_sup['원딜'] == carry) & \
                     (bot_sup['서폿'] == support)
        cond_count = condition.sum()
        if cond_count < 70: continue # 바텀 듀오 데이터가 100이하면 스킵

        stat = {}
        stat['count'] = cond_count
        stat['duoChamps'] = f'{carry}&{support}'
        stat['winRate'] = bot_sup.loc[condition, '승패'].sum() / cond_count
        duo_stats1 = pd.concat([duo_stats1, pd.Series(stat)], axis=1, sort=False)
duo_stats1 = duo_stats1.T.reset_index(drop=True)
#미드정글 조합
duo_stats2 = pd.DataFrame()
for carry in mid_jug['미드'].unique(): # 각 원딜과 서포터마다 계산
    for support in mid_jug['정글'].unique():
        condition =  (mid_jug['미드'] == carry) & \
                     (mid_jug['정글'] == support)
        cond_count = condition.sum()
        if cond_count < 30: continue # 바텀 듀오 데이터가 100이하면 스킵

        stat = {}
        stat['count'] = cond_count
        stat['duoChamps'] = f'{carry}&{support}'
        stat['winRate'] = mid_jug.loc[condition, '승패'].sum() / cond_count
        duo_stats2 = pd.concat([duo_stats2, pd.Series(stat)], axis=1, sort=False)
duo_stats2 = duo_stats2.T.reset_index(drop=True)
#탑정글 조합
duo_stats3 = pd.DataFrame()
for carry in top_jug['탑'].unique(): # 각 원딜과 서포터마다 계산
    for support in top_jug['정글'].unique():
        condition =  (top_jug['탑'] == carry) & \
                     (top_jug['정글'] == support)
        cond_count = condition.sum()
        if cond_count < 30: continue # 바텀 듀오 데이터가 100이하면 스킵

        stat = {}
        stat['count'] = cond_count
        stat['duoChamps'] = f'{carry}&{support}'
        stat['winRate'] = top_jug.loc[condition, '승패'].sum() / cond_count
        duo_stats3 = pd.concat([duo_stats3, pd.Series(stat)], axis=1, sort=False)
duo_stats3 = duo_stats3.T.reset_index(drop=True)


##### 탑정글의 승률 (30번 이상 쓰인조합)
탑과 정글의 조합에선 하드cc를 가진 정글챔피언과 폭딜을 넣을수있는 탑라이너간의 조합에서 높은 승률을 기록하였다.  
그리고 하드 cc기가 없거나 연계가 잘 되지않는 조합인 경우엔 낮은 승률을 기록한다.

In [117]:
duo_stats3.sort_values('winRate', ascending=False)

,count,duoChamps,winRate
29,34,Darius&Amumu,0.764706
48,32,Teemo&Trundle,0.71875
45,30,Camille&JarvanIV,0.7
17,50,Garen&MasterYi,0.66
77,32,Mordekaiser&XinZhao,0.65625
...,...,...,...
37,41,Nasus&Shaco,0.341463
58,37,Fiora&Kayn,0.324324
60,34,Fiora&LeeSin,0.323529
53,31,Teemo&LeeSin,0.290323


##### 미드정글의 승률 (30번 이상 쓰인조합)

미드와 정글의 조합에서도 서로의 cc기를 연계시킬수 있는조합과    
한쪽의 cc와 폭딜을 넣을 수 있는 조합이 높은 승률을 기록했다.  
미드와 탑 둘다 정글 리신의 티어가 매우낮다.

In [118]:
duo_stats2.sort_values('winRate', ascending=False)

,count,duoChamps,winRate
2,35,Yasuo&Trundle,0.742857
71,31,Annie&Khazix,0.677419
60,30,Ahri&JarvanIV,0.666667
48,35,Lux&Amumu,0.657143
22,34,Irelia&MasterYi,0.647059
...,...,...,...
58,35,Ahri&LeeSin,0.371429
44,38,Lux&Kayn,0.368421
68,30,Leblanc&XinZhao,0.366667
11,51,Yasuo&LeeSin,0.352941


##### 바텀듀오의 승률 (100번 이상 쓰인조합)

바텀은 많이나오는 챔피언 폭이 정해져있어서 100번 이상 쓰인조합을 확인해 보았다.  
서폿이나 원딜중 cc기를 한번 걸면 그다음 cc를 덮을수있는 조합이 고승률을 기록했다.

In [119]:
duo_stats1.sort_values('winRate', ascending=False)

,count,duoChamps,winRate
33,71,Ashe&Pyke,0.676056
30,81,Ashe&Sona,0.666667
27,70,Ashe&Blitzcrank,0.642857
7,74,Jhin&Yuumi,0.608108
26,140,Ashe&Lux,0.592857
11,78,Jhin&Xerath,0.589744
14,75,Vayne&Thresh,0.586667
15,74,Vayne&Yuumi,0.581081
31,73,Ashe&Nautilus,0.575342
1,79,Caitlyn&Brand,0.56962
